# Sentiment Analysis with an RNN

In this notebook, you'll implement a recurrent neural network that performs sentiment analysis. 
>Using an RNN rather than a strictly feedforward network is more accurate since we can include information about the *sequence* of words. 

Here we'll use a dataset of movie reviews, accompanied by sentiment labels: positive or negative.

<img src="assets/reviews_ex.png" width=40%>

### Network Architecture

The architecture for this network is shown below.

<img src="assets/network_diagram.png" width=40%>

>**First, we'll pass in words to an embedding layer.** We need an embedding layer because we have tens of thousands of words, so we'll need a more efficient representation for our input data than one-hot encoded vectors. You should have seen this before from the Word2Vec lesson. You can actually train an embedding with the Skip-gram Word2Vec model and use those embeddings as input, here. However, it's good enough to just have an embedding layer and let the network learn a different embedding table on its own. *In this case, the embedding layer is for dimensionality reduction, rather than for learning semantic representations.*

>**After input words are passed to an embedding layer, the new embeddings will be passed to LSTM cells.** The LSTM cells will add *recurrent* connections to the network and give us the ability to include information about the *sequence* of words in the movie review data. 

>**Finally, the LSTM outputs will go to a sigmoid output layer.** We're using a sigmoid function because positive and negative = 1 and 0, respectively, and a sigmoid will output predicted, sentiment values between 0-1. 

We don't care about the sigmoid outputs except for the **very last one**; we can ignore the rest. We'll calculate the loss by comparing the output at the last time step and the training label (pos or neg).

---
### Load in and visualize the data

In [1]:
import numpy as np

# read data from text files
with open('data/reviews.txt', 'r') as f:
    reviews = f.read()
with open('data/labels.txt', 'r') as f:
    labels = f.read()

In [2]:
print(reviews[:2000])
print()
print(labels[:20])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

## Data pre-processing

The first step when building a neural network model is getting your data into the proper form to feed into the network. Since we're using embedding layers, we'll need to encode each word with an integer. We'll also want to clean it up a bit.

You can see an example of the reviews data above. Here are the processing steps, we'll want to take:
>* We'll want to get rid of periods and extraneous punctuation.
* Also, you might notice that the reviews are delimited with newline characters `\n`. To deal with those, I'm going to split the text into each review using `\n` as the delimiter. 
* Then I can combined all the reviews back together into one big string.

First, let's remove all punctuation. Then get all the text without the newlines and split it into individual words.

In [3]:
from string import punctuation

print(punctuation)

# get rid of punctuation
reviews = reviews.lower() # lowercase, standardize
all_text = ''.join([c for c in reviews if c not in punctuation])

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [4]:
# split by new lines and spaces
reviews_split = all_text.split('\n')
all_text = ' '.join(reviews_split)

# create a list of words
words = all_text.split()

In [5]:
words[:30]

['bromwell',
 'high',
 'is',
 'a',
 'cartoon',
 'comedy',
 'it',
 'ran',
 'at',
 'the',
 'same',
 'time',
 'as',
 'some',
 'other',
 'programs',
 'about',
 'school',
 'life',
 'such',
 'as',
 'teachers',
 'my',
 'years',
 'in',
 'the',
 'teaching',
 'profession',
 'lead',
 'me']

### Encoding the words

The embedding lookup requires that we pass in integers to our network. The easiest way to do this is to create dictionaries that map the words in the vocabulary to integers. Then we can convert each of our reviews into integers so they can be passed into the network.

> **Exercise:** Now you're going to encode the words with integers. Build a dictionary that maps words to integers. Later we're going to pad our input vectors with zeros, so make sure the integers **start at 1, not 0**.
> Also, convert the reviews to integers and store the reviews in a new list called `reviews_ints`. 

In [6]:
# feel free to use this import 
from collections import Counter

word_count = Counter(words)
vocab = sorted(word_count, key=word_count.get, reverse=True)

## Build a dictionary that maps words to integers
vocab_to_int = {word: i for i, word in enumerate(vocab, 1)}

## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
reviews_ints = []
for review in reviews_split:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])
    

**Test your code**

As a text that you've implemented the dictionary correctly, print out the number of unique words in your vocabulary and the contents of the first, tokenized review.

In [7]:
# stats about vocabulary
print('Unique words: ', len((vocab_to_int)))  # should ~ 74000+
print()

# print tokens in first review
print('Tokenized review: \n', reviews_ints[:1])

Unique words:  74072

Tokenized review: 
 [[21025, 308, 6, 3, 1050, 207, 8, 2138, 32, 1, 171, 57, 15, 49, 81, 5785, 44, 382, 110, 140, 15, 5194, 60, 154, 9, 1, 4975, 5852, 475, 71, 5, 260, 12, 21025, 308, 13, 1978, 6, 74, 2395, 5, 613, 73, 6, 5194, 1, 24103, 5, 1983, 10166, 1, 5786, 1499, 36, 51, 66, 204, 145, 67, 1199, 5194, 19869, 1, 37442, 4, 1, 221, 883, 31, 2988, 71, 4, 1, 5787, 10, 686, 2, 67, 1499, 54, 10, 216, 1, 383, 9, 62, 3, 1406, 3686, 783, 5, 3483, 180, 1, 382, 10, 1212, 13583, 32, 308, 3, 349, 341, 2913, 10, 143, 127, 5, 7690, 30, 4, 129, 5194, 1406, 2326, 5, 21025, 308, 10, 528, 12, 109, 1448, 4, 60, 543, 102, 12, 21025, 308, 6, 227, 4146, 48, 3, 2211, 12, 8, 215, 23]]


### Encoding the labels

Our labels are "positive" or "negative". To use these labels in our network, we need to convert them to 0 and 1.

> **Exercise:** Convert labels from `positive` and `negative` to 1 and 0, respectively, and place those in a new list, `encoded_labels`.

In [8]:
# 1=positive, 0=negative label conversion
labels_split = labels.split('\n')
encoded_labels = [1 if label=='positive' else 0 for label in labels_split]


### Removing Outliers

As an additional pre-processing step, we want to make sure that our reviews are in good shape for standard processing. That is, our network will expect a standard input text size, and so, we'll want to shape our reviews into a specific length. We'll approach this task in two main steps:

1. Getting rid of extremely long or short reviews; the outliers
2. Padding/truncating the remaining data so that we have reviews of the same length.

<img src="assets/outliers_padding_ex.png" width=40%>

Before we pad our review text, we should check for reviews of extremely short or long lengths; outliers that may mess with our training.

In [9]:
# outlier review stats
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum review length: 2514


Okay, a couple issues here. We seem to have one review with zero length. And, the maximum review length is way too many steps for our RNN. We'll have to remove any super short reviews and truncate super long reviews. This removes outliers and should allow our model to train more efficiently.

> **Exercise:** First, remove *any* reviews with zero length from the `reviews_ints` list and their corresponding label in `encoded_labels`.

In [10]:
print('Number of reviews before removing outliers: ', len(reviews_ints))

## remove any reviews/labels with zero length from the reviews_ints list.
zero_idx = [i for i, length in enumerate(review_lens) if length==0]
reviews_ints = [review_i for i, review_i in enumerate(reviews_ints) if i not in zero_idx]
encoded_labels = [encoded_lbl for i, encoded_lbl in enumerate(encoded_labels) if i not in zero_idx]

print('Number of reviews after removing outliers: ', len(reviews_ints))

Number of reviews before removing outliers:  25001
Number of reviews after removing outliers:  25000


---
## Padding sequences

To deal with both short and very long reviews, we'll pad or truncate all our reviews to a specific length. For reviews shorter than some `seq_length`, we'll pad with 0s. For reviews longer than `seq_length`, we can truncate them to the first `seq_length` words. A good `seq_length`, in this case, is 200.

> **Exercise:** Define a function that returns an array `features` that contains the padded data, of a standard size, that we'll pass to the network. 
* The data should come from `review_ints`, since we want to feed integers to the network. 
* Each row should be `seq_length` elements long. 
* For reviews shorter than `seq_length` words, **left pad** with 0s. That is, if the review is `['best', 'movie', 'ever']`, `[117, 18, 128]` as integers, the row will look like `[0, 0, 0, ..., 0, 117, 18, 128]`. 
* For reviews longer than `seq_length`, use only the first `seq_length` words as the feature vector.

As a small example, if the `seq_length=10` and an input review is: 
```
[117, 18, 128]
```
The resultant, padded sequence should be: 

```
[0, 0, 0, 0, 0, 0, 0, 117, 18, 128]
```

**Your final `features` array should be a 2D array, with as many rows as there are reviews, and as many columns as the specified `seq_length`.**

This isn't trivial and there are a bunch of ways to do this. But, if you're going to be building your own deep learning networks, you're going to have to get used to preparing your data.

In [11]:
def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    
    # getting the correct rows x cols shape
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)

    # for each review, I grab that review and 
    for i, row in enumerate(reviews_ints):
        
        print(features[i, -len(row):].shape)
        features[i, -len(row):] = np.array(row)[:seq_length]
    return features

In [29]:
# Test your implementation!

seq_length = 200

features = pad_features(reviews_ints, seq_length=seq_length)

## test statements - do not change - ##
assert len(features)==len(reviews_ints), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 10 values of the first 30 batches 
print(features[:30,:10])

(140,)
(114,)
(200,)
(200,)
(150,)
(146,)
(124,)
(153,)
(121,)
(200,)
(184,)
(104,)
(112,)
(200,)
(200,)
(159,)
(200,)
(68,)
(200,)
(200,)
(200,)
(200,)
(86,)
(148,)
(200,)
(105,)
(200,)
(144,)
(148,)
(180,)
(162,)
(140,)
(196,)
(67,)
(200,)
(87,)
(132,)
(134,)
(155,)
(181,)
(200,)
(20,)
(200,)
(200,)
(177,)
(187,)
(186,)
(178,)
(144,)
(121,)
(106,)
(200,)
(109,)
(200,)
(200,)
(198,)
(150,)
(200,)
(145,)
(200,)
(200,)
(127,)
(122,)
(200,)
(166,)
(200,)
(168,)
(156,)
(200,)
(200,)
(200,)
(200,)
(56,)
(200,)
(200,)
(157,)
(200,)
(200,)
(200,)
(118,)
(200,)
(196,)
(167,)
(118,)
(200,)
(117,)
(200,)
(200,)
(111,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(89,)
(200,)
(200,)
(116,)
(200,)
(132,)
(200,)
(157,)
(185,)
(166,)
(200,)
(200,)
(200,)
(187,)
(200,)
(200,)
(185,)
(174,)
(163,)
(197,)
(57,)
(197,)
(200,)
(171,)
(98,)
(164,)
(184,)
(200,)
(134,)
(194,)
(200,)
(200,)
(200,)
(137,)
(168,)
(174,)
(110,)
(161,)
(200,)
(115,)
(111,)
(140,)
(96,)
(200,)
(126,)
(138,)
(196,)
(143,)
(2

(200,)
(107,)
(195,)
(125,)
(200,)
(123,)
(96,)
(178,)
(170,)
(157,)
(84,)
(200,)
(200,)
(200,)
(169,)
(191,)
(64,)
(136,)
(200,)
(200,)
(133,)
(116,)
(200,)
(159,)
(200,)
(200,)
(135,)
(158,)
(101,)
(200,)
(122,)
(196,)
(132,)
(200,)
(36,)
(200,)
(59,)
(200,)
(154,)
(196,)
(88,)
(132,)
(112,)
(93,)
(74,)
(138,)
(174,)
(126,)
(133,)
(133,)
(139,)
(154,)
(122,)
(200,)
(104,)
(158,)
(146,)
(200,)
(200,)
(158,)
(93,)
(136,)
(200,)
(200,)
(37,)
(200,)
(168,)
(200,)
(200,)
(132,)
(77,)
(200,)
(178,)
(176,)
(200,)
(140,)
(130,)
(147,)
(200,)
(200,)
(125,)
(200,)
(191,)
(140,)
(149,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(157,)
(200,)
(191,)
(200,)
(157,)
(155,)
(124,)
(74,)
(173,)
(140,)
(90,)
(200,)
(137,)
(174,)
(200,)
(200,)
(200,)
(200,)
(200,)
(131,)
(200,)
(190,)
(140,)
(98,)
(172,)
(200,)
(154,)
(114,)
(135,)
(200,)
(148,)
(89,)
(135,)
(200,)
(150,)
(197,)
(179,)
(200,)
(200,)
(178,)
(194,)
(177,)
(200,)
(113,)
(200,)
(53,)
(179,)
(95,)
(157,)
(51,)
(170,)
(71

(141,)
(200,)
(200,)
(163,)
(200,)
(200,)
(200,)
(130,)
(200,)
(200,)
(104,)
(200,)
(157,)
(195,)
(159,)
(144,)
(117,)
(132,)
(47,)
(37,)
(126,)
(200,)
(75,)
(200,)
(185,)
(73,)
(200,)
(99,)
(169,)
(104,)
(200,)
(159,)
(200,)
(87,)
(200,)
(200,)
(200,)
(200,)
(156,)
(84,)
(200,)
(45,)
(128,)
(52,)
(200,)
(200,)
(144,)
(137,)
(140,)
(193,)
(177,)
(200,)
(139,)
(200,)
(146,)
(123,)
(29,)
(186,)
(128,)
(200,)
(200,)
(200,)
(53,)
(110,)
(51,)
(188,)
(57,)
(55,)
(135,)
(135,)
(160,)
(70,)
(151,)
(200,)
(200,)
(200,)
(165,)
(200,)
(200,)
(63,)
(84,)
(200,)
(56,)
(95,)
(138,)
(81,)
(155,)
(200,)
(200,)
(200,)
(130,)
(98,)
(61,)
(135,)
(200,)
(157,)
(191,)
(200,)
(169,)
(164,)
(200,)
(154,)
(200,)
(200,)
(200,)
(200,)
(200,)
(195,)
(200,)
(150,)
(145,)
(169,)
(200,)
(143,)
(200,)
(124,)
(193,)
(200,)
(200,)
(200,)
(142,)
(124,)
(200,)
(173,)
(200,)
(139,)
(200,)
(183,)
(150,)
(134,)
(200,)
(200,)
(200,)
(152,)
(65,)
(130,)
(200,)
(100,)
(200,)
(200,)
(200,)
(200,)
(200,)
(130,)
(132,)
(187,)
(

(200,)
(200,)
(200,)
(176,)
(175,)
(148,)
(109,)
(178,)
(200,)
(129,)
(200,)
(146,)
(151,)
(115,)
(115,)
(71,)
(200,)
(127,)
(200,)
(200,)
(150,)
(200,)
(185,)
(90,)
(81,)
(152,)
(33,)
(125,)
(174,)
(140,)
(200,)
(147,)
(98,)
(154,)
(188,)
(114,)
(95,)
(130,)
(121,)
(125,)
(119,)
(131,)
(129,)
(200,)
(200,)
(200,)
(91,)
(200,)
(118,)
(200,)
(106,)
(200,)
(200,)
(131,)
(112,)
(160,)
(200,)
(186,)
(200,)
(200,)
(200,)
(108,)
(200,)
(200,)
(192,)
(200,)
(132,)
(200,)
(200,)
(180,)
(200,)
(200,)
(155,)
(200,)
(200,)
(200,)
(131,)
(200,)
(200,)
(200,)
(129,)
(130,)
(200,)
(77,)
(63,)
(200,)
(121,)
(165,)
(172,)
(200,)
(132,)
(200,)
(186,)
(200,)
(27,)
(200,)
(62,)
(200,)
(134,)
(176,)
(125,)
(164,)
(200,)
(200,)
(200,)
(63,)
(114,)
(170,)
(200,)
(200,)
(200,)
(200,)
(148,)
(200,)
(185,)
(60,)
(185,)
(72,)
(185,)
(72,)
(200,)
(45,)
(156,)
(142,)
(130,)
(200,)
(64,)
(136,)
(119,)
(200,)
(200,)
(200,)
(148,)
(200,)
(132,)
(188,)
(200,)
(190,)
(200,)
(146,)
(167,)
(117,)
(135,)
(200,)
(200,)
(2

(200,)
(139,)
(158,)
(200,)
(143,)
(200,)
(186,)
(188,)
(200,)
(197,)
(124,)
(200,)
(129,)
(139,)
(200,)
(200,)
(146,)
(147,)
(200,)
(200,)
(114,)
(184,)
(133,)
(200,)
(200,)
(200,)
(195,)
(130,)
(200,)
(138,)
(130,)
(200,)
(195,)
(51,)
(57,)
(145,)
(126,)
(200,)
(200,)
(117,)
(200,)
(200,)
(177,)
(200,)
(200,)
(132,)
(154,)
(200,)
(200,)
(159,)
(153,)
(200,)
(145,)
(200,)
(137,)
(200,)
(75,)
(200,)
(136,)
(125,)
(122,)
(144,)
(200,)
(200,)
(200,)
(164,)
(200,)
(126,)
(200,)
(141,)
(145,)
(130,)
(200,)
(200,)
(200,)
(200,)
(176,)
(127,)
(158,)
(200,)
(79,)
(120,)
(132,)
(200,)
(164,)
(200,)
(121,)
(200,)
(158,)
(200,)
(161,)
(114,)
(137,)
(96,)
(182,)
(200,)
(141,)
(200,)
(148,)
(200,)
(88,)
(166,)
(200,)
(200,)
(185,)
(200,)
(61,)
(190,)
(200,)
(111,)
(171,)
(200,)
(171,)
(156,)
(139,)
(133,)
(137,)
(200,)
(148,)
(200,)
(151,)
(156,)
(200,)
(200,)
(126,)
(173,)
(128,)
(142,)
(170,)
(200,)
(200,)
(200,)
(67,)
(117,)
(162,)
(200,)
(191,)
(200,)
(166,)
(115,)
(200,)
(200,)
(145,)
(130,)


...

(200,)
(73,)
(200,)
(153,)
(146,)
(200,)
(200,)
(200,)
(200,)
(166,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(169,)
(181,)
(111,)
(113,)
(144,)
(138,)
(167,)
(200,)
(200,)
(152,)
(200,)
(200,)
(153,)
(200,)
(200,)
(200,)
(200,)
(174,)
(200,)
(160,)
(71,)
(123,)
(200,)
(200,)
(200,)
(200,)
(175,)
(200,)
(111,)
(145,)
(128,)
(190,)
(116,)
(140,)
(137,)
(147,)
(200,)
(44,)
(140,)
(179,)
(122,)
(143,)
(200,)
(152,)
(200,)
(90,)
(200,)
(200,)
(200,)
(167,)
(200,)
(126,)
(200,)
(172,)
(145,)
(100,)
(200,)
(118,)
(200,)
(200,)
(161,)
(178,)
(177,)
(200,)
(200,)
(177,)
(200,)
(141,)
(200,)
(160,)
(128,)
(129,)
(101,)
(200,)
(82,)
(173,)
(68,)
(164,)
(107,)
(191,)
(117,)
(200,)
(169,)
(200,)
(160,)
(131,)
(120,)
(186,)
(200,)
(200,)
(200,)
(119,)
(200,)
(200,)
(200,)
(200,)
(200,)
(148,)
(160,)
(156,)
(200,)
(200,)
(200,)
(200,)
(127,)
(168,)
(200,)
(152,)
(200,)
(200,)
(99,)
(186,)
(187,)
(166,)
(117,)
(193,)
(200,)
(152,)
(196,)
(200,)
(118,)
(200,)
(200,)
(200,)
(200,)
(78,)


(200,)
(104,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(56,)
(200,)
(200,)
(200,)
(99,)
(200,)
(200,)
(196,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(135,)
(200,)
(99,)
(200,)
(99,)
(200,)
(103,)
(195,)
(183,)
(123,)
(104,)
(200,)
(65,)
(100,)
(60,)
(200,)
(158,)
(72,)
(90,)
(200,)
(43,)
(200,)
(124,)
(52,)
(200,)
(67,)
(182,)
(148,)
(120,)
(200,)
(184,)
(200,)
(139,)
(200,)
(171,)
(200,)
(151,)
(161,)
(62,)
(200,)
(192,)
(133,)
(200,)
(200,)
(200,)
(146,)
(155,)
(200,)
(168,)
(169,)
(118,)
(152,)
(164,)
(179,)
(160,)
(156,)
(200,)
(192,)
(177,)
(163,)
(200,)
(200,)
(179,)
(200,)
(150,)
(129,)
(200,)
(200,)
(200,)
(149,)
(105,)
(200,)
(200,)
(200,)
(200,)
(177,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(124,)
(139,)
(106,)
(200,)
(167,)
(165,)
(200,)
(200,)
(172,)
(186,)
(200,)
(171,)
(164,)
(200,)
(199,)
(200,)
(137,)
(200,)
(191,)
(200,)
(180,)
(200,)
(200,)
(183,)
(200,)
(194,)
(200,)
(186,)
(143,)
(200,)
(122,)
(189,)
(122,)
(200,)
(159,)
(200,)
(200,)
(200,)
(183

(200,)
(200,)
(200,)
(200,)
(164,)
(118,)
(200,)
(140,)
(200,)
(171,)
(137,)
(135,)
(200,)
(148,)
(200,)
(125,)
(200,)
(115,)
(198,)
(200,)
(43,)
(200,)
(65,)
(138,)
(200,)
(156,)
(200,)
(200,)
(200,)
(200,)
(88,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(84,)
(200,)
(142,)
(138,)
(123,)
(200,)
(164,)
(200,)
(135,)
(200,)
(155,)
(105,)
(175,)
(200,)
(114,)
(60,)
(200,)
(200,)
(200,)
(69,)
(200,)
(200,)
(127,)
(138,)
(176,)
(115,)
(200,)
(52,)
(200,)
(155,)
(200,)
(126,)
(200,)
(49,)
(200,)
(110,)
(188,)
(200,)
(19,)
(152,)
(140,)
(179,)
(200,)
(134,)
(200,)
(200,)
(200,)
(159,)
(200,)
(171,)
(65,)
(157,)
(151,)
(129,)
(200,)
(172,)
(176,)
(170,)
(200,)
(200,)
(200,)
(97,)
(82,)
(140,)
(124,)
(200,)
(171,)
(102,)
(66,)
(163,)
(115,)
(63,)
(200,)
(200,)
(200,)
(128,)
(152,)
(54,)
(196,)
(200,)
(166,)
(200,)
(200,)
(200,)
(200,)
(169,)
(99,)
(152,)
(200,)
(161,)
(200,)
(200,)
(200,)
(129,)
(80,)
(134,)
(200,)
(149,)
(200,)
(141,)
(70,)
(200,)
(79,)
(131,)
(171,)
(136,)
(200,)
(129,)
(200

(169,)
(136,)
(153,)
(200,)
(193,)
(164,)
(131,)
(200,)
(200,)
(196,)
(200,)
(200,)
(174,)
(200,)
(200,)
(169,)
(200,)
(94,)
(125,)
(200,)
(200,)
(200,)
(117,)
(142,)
(200,)
(92,)
(200,)
(153,)
(163,)
(200,)
(141,)
(85,)
(100,)
(30,)
(188,)
(124,)
(51,)
(85,)
(84,)
(175,)
(158,)
(200,)
(200,)
(200,)
(112,)
(124,)
(193,)
(200,)
(47,)
(200,)
(160,)
(200,)
(153,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(188,)
(200,)
(200,)
(200,)
(56,)
(95,)
(200,)
(195,)
(44,)
(200,)
(161,)
(150,)
(161,)
(200,)
(200,)
(106,)
(200,)
(200,)
(85,)
(200,)
(200,)
(200,)
(200,)
(138,)
(123,)
(200,)
(200,)
(140,)
(200,)
(200,)
(87,)
(200,)
(129,)
(169,)
(148,)
(151,)
(81,)
(200,)
(141,)
(115,)
(90,)
(200,)
(200,)
(200,)
(128,)
(200,)
(200,)
(108,)
(190,)
(200,)
(188,)
(167,)
(188,)
(150,)
(200,)
(76,)
(154,)
(200,)
(198,)
(200,)
(200,)
(200,)
(160,)
(200,)
(200,)
(200,)
(200,)
(135,)
(145,)
(150,)
(200,)
(178,)
(105,)
(200,)
(121,)
(200,)
(129,)
(68,)
(119,)
(155,)
(200,)
(200,)
(129,)
(138,)
(197,)
(1

ValueError: could not broadcast input array from shape (0) into shape (200)

## Training, Validation, Test

With our data in nice shape, we'll split it into training, validation, and test sets.

> **Exercise:** Create the training, validation, and test sets. 
* You'll need to create sets for the features and the labels, `train_x` and `train_y`, for example. 
* Define a split fraction, `split_frac` as the fraction of data to **keep** in the training set. Usually this is set to 0.8 or 0.9. 
* Whatever data is left will be split in half to create the validation and *testing* data.

In [24]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(features)*0.8)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

NameError: name 'features' is not defined

**Check your work**

With train, validation, and test fractions equal to 0.8, 0.1, 0.1, respectively, the final, feature data shapes should look like:
```
                    Feature Shapes:
Train set: 		 (20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		  (2500, 200)
```

---
## DataLoaders and Batching

After creating training, test, and validation data, we can create DataLoaders for this data by following two steps:
1. Create a known format for accessing our data, using [TensorDataset](https://pytorch.org/docs/stable/data.html#) which takes in an input set of data and a target set of data with the same first dimension, and creates a dataset.
2. Create DataLoaders and batch our training, validation, and test Tensor datasets.

```
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
train_loader = DataLoader(train_data, batch_size=batch_size)
```

This is an alternative to creating a generator function for batching our data into full batches.

In [18]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50

# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

NameError: name 'train_x' is not defined

In [13]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

NameError: name 'train_loader' is not defined

---
# Sentiment Network with PyTorch

Below is where you'll define the network.

<img src="assets/network_diagram.png" width=40%>

The layers are as follows:
1. An [embedding layer](https://pytorch.org/docs/stable/nn.html#embedding) that converts our word tokens (integers) into embeddings of a specific size.
2. An [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) defined by a hidden_state size and number of layers
3. A fully-connected output layer that maps the LSTM layer outputs to a desired output_size
4. A sigmoid activation layer which turns all outputs into a value 0-1; return **only the last sigmoid output** as the output of this network.

### The Embedding Layer

We need to add an [embedding layer](https://pytorch.org/docs/stable/nn.html#embedding) because there are 74000+ words in our vocabulary. It is massively inefficient to one-hot encode that many classes. So, instead of one-hot encoding, we can have an embedding layer and use that layer as a lookup table. You could train an embedding layer using Word2Vec, then load it here. But, it's fine to just make a new layer, using it for only dimensionality reduction, and let the network learn the weights.


### The LSTM Layer(s)

We'll create an [LSTM](https://pytorch.org/docs/stable/nn.html#lstm) to use in our recurrent network, which takes in an input_size, a hidden_dim, a number of layers, a dropout probability (for dropout between multiple layers), and a batch_first parameter.

Most of the time, you're network will have better performance with more layers; between 2-3. Adding more layers allows the network to learn really complex relationships. 

> **Exercise:** Complete the `__init__`, `forward`, and `init_hidden` functions for the SentimentRNN model class.

Note: `init_hidden` should initialize the hidden and cell state of an lstm layer to all zeros, and move those state to GPU, if available.

In [14]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

NameError: name 'torch' is not defined

In [ ]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # define all layers
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        
        return hidden
        

## Instantiate the network

Here, we'll instantiate the network. First up, defining the hyperparameters.

* `vocab_size`: Size of our vocabulary or the range of values for our input, word tokens.
* `output_size`: Size of our desired output; the number of class scores we want to output (pos/neg).
* `embedding_dim`: Number of columns in the embedding lookup table; size of our embeddings.
* `hidden_dim`: Number of units in the hidden layers of our LSTM cells. Usually larger is better performance wise. Common values are 128, 256, 512, etc.
* `n_layers`: Number of LSTM layers in the network. Typically between 1-3

> **Exercise:** Define the model  hyperparameters.


In [ ]:
# Instantiate the model w/ hyperparams
vocab_size = 
output_size = 
embedding_dim = 
hidden_dim = 
n_layers = 

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

---
## Training

Below is the typical training code. If you want to do this yourself, feel free to delete all this code and implement it yourself. You can also add code to save a model by name.

>We'll also be using a new kind of cross entropy loss, which is designed to work with a single Sigmoid output. [BCELoss](https://pytorch.org/docs/stable/nn.html#bceloss), or **Binary Cross Entropy Loss**, applies cross entropy loss to a single value between 0 and 1.

We also have some data and training hyparameters:

* `lr`: Learning rate for our optimizer.
* `epochs`: Number of times to iterate through the training dataset.
* `clip`: The maximum gradient value to clip at (to prevent exploding gradients).

In [ ]:
# loss and optimization functions
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)


In [ ]:
# training params

epochs = 4 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

---
## Testing

There are a few ways to test your network.

* **Test data performance:** First, we'll see how our trained model performs on all of our defined test_data, above. We'll calculate the average loss and accuracy over the test data.

* **Inference on user-generated data:** Second, we'll see if we can input just one example review at a time (without a label), and see what the trained model predicts. Looking at new, user input data like this, and predicting an output label, is called **inference**.

In [ ]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

### Inference on a test review

You can change this test_review to any text that you want. Read it and think: is it pos or neg? Then see if your model predicts correctly!
    
> **Exercise:** Write a `predict` function that takes in a trained net, a plain text_review, and a sequence length, and prints out a custom statement for a positive or negative review!
* You can use any functions that you've already defined or define any helper functions you want to complete `predict`, but it should just take in a trained net, a text review, and a sequence length.


In [ ]:
# negative test review
test_review_neg = 'The worst movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow.'


In [ ]:
def predict(net, test_review, sequence_length=200):
    ''' Prints out whether a give review is predicted to be 
        positive or negative in sentiment, using a trained model.
        
        params:
        net - A trained net 
        test_review - a review made of normal text and punctuation
        sequence_length - the padded length of a review
        '''
    
    
    # print custom response based on whether test_review is pos/neg
    
        

In [ ]:
# positive test review
test_review_pos = 'This movie had the best acting and the dialogue was so good. I loved it.'


In [ ]:
# call function
# try negative and positive reviews!
seq_length=200
predict(net, test_review_neg, seq_length)

### Try out test_reviews of your own!

Now that you have a trained model and a predict function, you can pass in _any_ kind of text and this model will predict whether the text has a positive or negative sentiment. Push this model to its limits and try to find what words it associates with positive or negative.

Later, you'll learn how to deploy a model like this to a production environment so that it can respond to any kind of user data put into a web app!